In [2]:
import numpy as np
import pandas as pd
import os
import glob
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import joblib
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [11]:
data_train = pd.read_csv("can to decan data/ECMO_data_clustered_train.csv")

In [12]:
data_test = pd.read_csv("can to decan data/ECMO_data_clustered_test.csv")

In [19]:
df = pd.concat([data_train, data_test], axis=0)

In [20]:
df

,temperature,map_line,map_cuff,pulse,unassisted_resp_rate,end_tidal_co2,o2_flow_rate,base_excess,bicarb_(hco3),blood_urea_nitrogen_(bun),...,erythrocyte_sedimentation_rate_(esr),gcs_total_score,best_map,pf_sp,pf_pa,spo2,partial_pressure_of_oxygen_(pao2),rass_score,CAM_ICU,csn
0,1.654985,1.543293,0.712217,1.037490,-0.277493,1.443907,-0.794470,-0.319874,-0.824496,0.070542,...,1.634933,-1.598843,1.475408,1.028444,-1.108899,-0.454683,-0.687484,-1.002158,1.311655,43640203
1,0.179388,0.036551,0.685907,-0.148586,-0.277493,1.223377,-0.794470,-0.319874,-0.880290,0.051429,...,1.634933,-1.598843,-0.011264,-1.524705,-1.093262,-0.158852,-0.640286,-0.579654,1.311655,43640203
2,-1.182702,0.325409,0.685907,-0.125951,-0.277493,1.100080,-0.794470,-0.319874,-0.746384,0.032315,...,1.634933,-1.598843,0.273745,-1.533378,-1.065115,-0.333475,-0.555330,-0.438819,1.311655,43640203
3,-0.501657,-0.420155,-0.300701,1.405686,-0.312486,0.691097,-0.794470,-0.319874,-1.023762,0.032315,...,1.634933,-0.534057,-0.461888,-1.533889,-1.087007,-0.343746,-0.621407,-0.016316,-0.279635,43640203
4,1.314463,-0.100070,0.093942,3.125949,-0.067534,0.817400,-0.794470,-0.319874,-0.387706,0.032315,...,1.634933,0.530729,0.115835,-0.819034,-0.562856,0.858066,-0.064474,1.673699,-0.810065,43640203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3379,1.087448,-0.623136,-0.037606,1.828207,0.387375,-0.472703,-0.628213,0.397329,0.702675,-0.617543,...,-0.428030,1.240587,-0.662165,1.306515,1.020870,-0.425922,1.124907,1.251195,-0.810065,70454151283
3380,0.803679,0.013131,-0.037606,1.164246,0.212410,-0.472703,-0.628213,0.397329,0.702675,-0.617543,...,-0.428030,1.240587,-0.034373,1.306515,1.020870,0.549909,1.124907,0.969526,-0.810065,70454151283
3381,0.917187,-0.291341,-0.037606,1.383051,0.527347,-0.472703,-0.628213,0.397329,0.893970,-0.923358,...,-0.428030,1.240587,-0.334789,1.306515,1.020870,0.703988,1.124907,0.969526,-0.810065,70454151283
3382,0.349649,-0.634846,-0.037606,0.555615,-0.067534,-0.472703,-0.628213,0.397329,1.467855,-0.923358,...,-0.428030,1.240587,-0.673720,1.306515,1.020870,0.380423,1.124907,0.687857,-0.810065,70454151283


In [21]:
csn_counts = df['csn'].value_counts()

# Calculate the average number of occurrences
average_occurrences = csn_counts.mean()

# Calculate the standard deviation of occurrences
std_occurrences = csn_counts.std()

In [22]:
average_occurrences

82.56521739130434

In [23]:
std_occurrences

91.69786505524648